<a href="https://colab.research.google.com/github/dieko95/blackouts-C4V/blob/diego-first-iter/twitter_pretagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Dataset Creation - Tagging 

This notebook aims to create the dataset for Code for Venezuela's Blackout Project. 

This dataset is going to be consumed by an ML model that will aim to predict: 

- If a tweet is from Venezuela
- If so from which state(s) 
- About what public service the user is reporting (sinluz)



## Libraries



In [0]:
import pandas as pd
import re

# For better visualization of text in Pandas DF
pd.set_option('display.max_colwidth', None)

## Accessing Data

The untagged dataset originates from scraped tweets by Code For Venezuela's Angostura ETL. A subset of tweets (11,000) was queried from the etl in order for them to be tagged. The first 4,000 tweets have already been tagged. 

In [0]:
# Read CSV from github 
tagOriginalDf = pd.read_csv('https://raw.githubusercontent.com/dieko95/blackouts-C4V/diego-first-iter/tagging-set-original_for_jupyter_tagging.csv')

# Corresponding Section
tags_df = tagOriginalDf.iloc[6001:8499,:].copy()

# Tagged Tweets
pre_tag_df = tags_df[['full_text','concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[:271,:].copy()

# Tweets to tag
to_tag_df = tags_df[['full_text','concat_text_user_description', 'label_country', 'label_state', 'label_type']].iloc[271:,:].copy()

## Cleaning Text

This is a helper function to quickly clean text.

- Converts all text to low caps. 
- Strips all spanish accents

Pending:

- Strip dots and links (@ and # must remain) 

In [0]:
def cleaner(df,text_col):
  # to lower

  df[text_col] = df[text_col].str.lower()

  # Convert common spanish accents

  df[text_col] = df[text_col].str.replace("ú", "u")
  df[text_col] = df[text_col].str.replace("ù", "u")
  df[text_col] = df[text_col].str.replace("ü", "u")
  df[text_col] = df[text_col].str.replace("ó", "o")
  df[text_col] = df[text_col].str.replace("ò", "o")
  df[text_col] = df[text_col].str.replace("í", "i")
  df[text_col] = df[text_col].str.replace("ì", "i")
  df[text_col] = df[text_col].str.replace("é", "e")
  df[text_col] = df[text_col].str.replace("è", "e")
  df[text_col] = df[text_col].str.replace("á", "a")
  df[text_col] = df[text_col].str.replace("à", "a")
  df[text_col] = df[text_col].str.replace("ñ", "gn")

  return df


to_tag_df = cleaner(to_tag_df, 'concat_text_user_description')
to_tag_df = cleaner(to_tag_df, 'full_text')

## Sections to Tag 

- Tag label_type (service reported)
  - Extracting pound signs (\#)

- Tag Country
  - Matches any state? 
  - has keyword 'edo' or 'estado' in it?
  - Follows any of the common accounts?
- Tag State
  - Match with list of venezuela states
  - We can use a list of venezuelan cities as well 

### Classifying Label Type

#### Hashtags

* \#SinLuz






In [0]:

# sinluz         670
# ahora           68
# sinagua         66
# apagon          65
# singasolina     53

hashtags = pd.Series(re.findall('#(\w+)', to_tag_df.concat_text_user_description.to_string())).copy()

# hashtags.value_counts()[hashtags.value_counts() > 10]

# hashtags[hashtags == 'sinl']



### Tagging Country

  - Matches any state? 
  - has keyword 'edo' or 'estado' in it?
  - Follows any of the common accounts?

*Notes*
  - For this section I will use the tweet's original text. If I include the user description it can add noise because a user can be reporting about a power outage of another state (e.g., I'm from caracas and reporting a power outage in Zulia)
  

In [45]:
# Read csv with Venezuela's administrative distribution
geo_df = pd.read_csv('/content/drive/My Drive/monitor_ciudad/cod.csv')

# Clean Columns
cols = ['parroquia', 'nombrepob', 'estado','nom_mun']

for col in cols:
  geo_df = cleaner(geo_df, col)

# # Unique pob name
#   # CHECK ENCODING ERROR WITH JESUS  carupano
# # geo_df.nombrepob.unique()

# Read states
state = geo_df.estado.unique().tolist()

# Read Municipalities
muni = geo_df.nom_mun.unique().tolist()

# Which indices contain any state 
indices = to_tag_df.full_text.str.contains('|'.join(state), 
                                                              case = False) # 6754

# Which indices contain "edo"
# indices = to_tag_df.full_text.str.contains('\sedo', case = False)

## View results
to_tag_df[indices]

## Tag Venezuela 
to_tag_df.loc[indices,'label_country'] = 'venezuela'



## Quality Control 



# tst = 'Todavía #SinLuz y son las 6:30  pm nueve horas sin luz en San Cristóbal  estado tachira sector los #kioskos #SinLuz vía seguro social exigimos el servicio ya dejen la burla y el cinismo al pueblo hijos de putas @CORPOELECinfo @corpoelectachir @NicolasMaduro @dcabellor "Luis Suarez" "LuisSua45172692" "soy abogado egresado de la universidad Javeriana de Bogotá Colombia  especialista en derecho internacional"'
# tst = cleaner(pd.DataFrame({'col':[tst]}), 'col').to_string()
# # If there's more than one element in state then it's multiple
# for i in state:
#   if i in tst:
#     print(i)
#   else:
#     next




[6274, 6275, 6326, 6327, 6567, 6599, 6618, 6670, 6694, 6708, 6805, 6822, 7190, 7238, 7262, 7264, 7299, 7357, 7399, 7497, 7524, 7526, 7602, 7630, 7666, 7679, 7724, 7756, 7791, 7792, 7801, 7853, 7878, 7881, 7903, 7925, 8011, 8031, 8036, 8054, 8090, 8122, 8153, 8188, 8355, 8356, 8427, 8491]


#### Municipality

In [0]:

## There's a municipality called
  # Democracy 

# indices = to_tag_df.concat_text_user_description.str.contains('|'.join(muni), case = False) # 6754

# to_tag_df[indices]

In [47]:
# 653 tweets with hashtags Out of 2228 tweets



indices = to_tag_df.concat_text_user_description.str.contains('venezuela', case = False) # 6754

to_tag_df[indices]

,full_text,concat_text_user_description,label_country,label_state,label_type
6273,"tenemos reportes de apagones por el area de maracay, venezuela. | reporta tu apagon con #sinluz","tenemos reportes de apagones por el area de maracay, venezuela. | reporta tu apagon con #sinluz ""sinluzvenezuela"" ""sefuelaluz_vzla"" ""verifying #sinluz around twitter in venezuela""",Venezuela,0,
6279,"hace como media hora llego la electricidad a el parral, valencia. mas de 8 horas estuvimos #sinluz. @corpoeleccar_ @rafaellacava10","hace como media hora llego la electricidad a el parral, valencia. mas de 8 horas estuvimos #sinluz. @corpoeleccar_ @rafaellacava10 ""mayte rizzo"" ""mayterizzo"" ""profesional universitaria. #anticomunista. sin fe no hay esperanza, y sin esperanza no hay amor. ¡dios, bendice a venezuela!""",0,0,
6282,"#venezuela el coronareginen del siglo xxi #sinluz #singasolina #sinpatria #sininternet #sinagua,sin gas,s/ medicinas,sin comida,s/vida digna, los miembros d la banda de la mafia roja, 21 agnos de corrupcion, miseria y muerte, son el estigma de la peor herencia de la humanidad","#venezuela el coronareginen del siglo xxi #sinluz #singasolina #sinpatria #sininternet #sinagua,sin gas,s/ medicinas,sin comida,s/vida digna, los miembros d la banda de la mafia roja, 21 agnos de corrupcion, miseria y muerte, son el estigma de la peor herencia de la humanidad ""sandra d."" ""sandracd10"" ""periodista y ucabista""",0,0,
6289,conductores hacen largas colas en caracas para surtirse de combustible https://t.co/awvt6trfco\n\n#singasolina #nacional #venevision #venezuela #noticierovenevision https://t.co/a3wv51xnng,"conductores hacen largas colas en caracas para surtirse de combustible https://t.co/awvt6trfco\n\n#singasolina #nacional #venevision #venezuela #noticierovenevision https://t.co/a3wv51xnng ""noticiero venevision"" ""noticierovv"" ""programa informativo venezolano que difunde sus noticias por tv a traves de venevision, y por su sitio web http://t.co/aqzltxrumf""",0,0,
6297,"1:22 am se restablece el servicio electrico en la ciudad de san juan de los morros, luego del #apagonnacional ocurrido aproximadamente a las 4:00 pm #sinluz #23jul","1:22 am se restablece el servicio electrico en la ciudad de san juan de los morros, luego del #apagonnacional ocurrido aproximadamente a las 4:00 pm #sinluz #23jul ""deivis gutierrez"" ""deivis_dg1"" ""estudiante de comunicacion social (unerg) la pasion por el periodismo me motiva a luchar por mi pais instagram: @deivis_dg1 venezuela""",0,0,
...,...,...,...,...,...
8479,#3ago | apagones afectan tambien a la industria del calzado &gt;&gt; https://t.co/e3dsfupyvr #reporteelectrico #sinluz #venezuela https://t.co/oilb3ero09,"#3ago | apagones afectan tambien a la industria del calzado &gt;&gt; https://t.co/e3dsfupyvr #reporteelectrico #sinluz #venezuela https://t.co/oilb3ero09 ""reporte electrico"" ""reportelectrico"" ""cuenta dedicada a informar sobre la crisis electrica de venezuela y otras noticias de relevancia. tu aporte es importante siguenos #reporteelectrico""",0,0,
8480,#sinluz #apagon #venezuela #apagonnacional #electromagnetico aqui el culpable del ataque #guayana https://t.co/vymfv7bqhr,"#sinluz #apagon #venezuela #apagonnacional #electromagnetico aqui el culpable del ataque #guayana https://t.co/vymfv7bqhr ""cesar a"" ""cesarav7"" ""dios ,hija & flia debemos buscar en los demas la felicidad que deseamos para nosotros mismos,d l, namaste ,community manager, mercadologo 5.0/social media👽""",0,0,
8483,"#29oct 8:20 pm #barinas\n\n sin servicio electrico comunidades guanapa, 5 julio, la paz, la esmeralda, redoma industrial. \n\n14 dias sin suministro de agua potable.\n\n✝️ socialismo asesino ✝️\n\n@hidroandesca @codbarinas @gobarinas \n#sinagua\n#sinluz\n@reporteya @periodistadsd","#29oct 8:20 pm #barinas\n\n sin servicio electrico comunidades guanapa, 5 julio, la paz, la esmeralda, redoma industrial. \n\n14 dias sin suministro de agua potable.\n\n✝️ socialismo asesino ✝️\n\n@hidroandesca

## Tagging State


Incluimos cuentas que reportan a nivel nacional? Es ruido porque lo que hacen es repetir lo que otros usuarios dicen? O captura señal porque son reportes de fallas de luz?

~~~
print(tags_df.loc[8122,'full_text'])

#Ahora Reportan más zonas #SinLuz: 

Catia, Distrito Capital ❌💡
Guatire y Guarenas, Edo. Miranda ❌💡
Estado Mérida ❌💡
Estado Aragua ❌💡

Comenta si hay fallas en tu zona #2Oct

~~~

In [69]:

_multiple = []
_single = []
for index,row in to_tag_df.loc[to_tag_df.label_country == 'venezuela', 'full_text'].iteritems():
  _lst = []

  for state_name in state:

    if state_name in row:
      _lst.append(1)
    

      if len(_lst) == 1:
        _single.append(index)

      elif len(_lst) > 1:
        _multiple.append(index)

      else:
        next


      print(len(_lst), '------', index)
    else:
      next

print(_single)

# tst = to_tag_df.loc[8122,'full_text']
# tst = cleaner(pd.DataFrame({'col':[tst]}), 'col').to_string()
# # If there's more than one element in state then it's multiple
# for i in state:
#   if i in tst:
#     print(i)
#   else:
#     next


1 ------ 6274
1 ------ 6275
1 ------ 6326
1 ------ 6327
1 ------ 6567
1 ------ 6599
1 ------ 6618
1 ------ 6670
1 ------ 6694
1 ------ 6708
1 ------ 6805
1 ------ 6822
1 ------ 7190
1 ------ 7238
1 ------ 7262
1 ------ 7264
1 ------ 7299
1 ------ 7357
1 ------ 7399
1 ------ 7497
1 ------ 7524
1 ------ 7526
1 ------ 7602
1 ------ 7630
1 ------ 7666
1 ------ 7679
1 ------ 7724
2 ------ 7724
1 ------ 7756
1 ------ 7791
2 ------ 7791
1 ------ 7792
1 ------ 7801
1 ------ 7853
1 ------ 7878
1 ------ 7881
1 ------ 7903
1 ------ 7925
1 ------ 8011
2 ------ 8011
1 ------ 8031
1 ------ 8036
1 ------ 8054
1 ------ 8090
1 ------ 8122
2 ------ 8122
3 ------ 8122
4 ------ 8122
1 ------ 8153
2 ------ 8153
1 ------ 8188
1 ------ 8355
1 ------ 8356
1 ------ 8427
1 ------ 8491
[6274, 6275, 6326, 6327, 6567, 6599, 6618, 6670, 6694, 6708, 6805, 6822, 7190, 7238, 7262, 7264, 7299, 7357, 7399, 7497, 7524, 7526, 7602, 7630, 7666, 7679, 7724, 7756, 7791, 7792, 7801, 7853, 7878, 7881, 7903, 7925, 8011, 8031, 8